### Problem Statement

This project was focused around how Big Mountain Resort (BMR) can capitalize more at its facilities as it expands and successfully grow its business by implementing a new price model for its ski resort tickets. This project aimed to build a predictive model for ticket prices based on the features and excursions present at a ski resort. The data for building this model included information on 330 resorts in the US that can be considered part of BMR same market share and were supplied by BMR Database Manager, as well as geographical and population demographics available scraped from Wikipedia.

### Data Wrangling

The `AdultWeekend` feature was chosen as the target column ahead of the `AdultWeekday` as it contained more entries. Although Montana was on the lower end when the median `AdultWeekday` and `AdultWeekday` prices were compared among the states, BMR was the outlier pointed out in the image below.

![plot comparing median adult weekday and adult weekend ticket prices among states pointing out BMR position](../images/median_ticket_prices.svg)

With the target column `AdultWeekend`, we were missing ticket prices for 16% of the resorts. This was a fundamental problem since we lack the required target data for those records.

More than 50% of the records had null values for the `fastEight` feature, which represents the number of fast 8 person chairs so that feature was dropped.

A few outliers were spotted with some of the numerical features. 
- `SkiableTerrain_ac` which represented the total scalable area in square acres had a value `> 26,000` for one of the records.This outlier was fact checked on that resort's website and a "*data correction*" was made. The true value was exactly 25,000 less.
- `yearsOpen` which represents the total number of years that the resort has been opened had a value of 2019 for one of the records. This was most likely input error: perhaps that resort opened in 2019. However, this row was dropped as assuming that it opened in 2019 would have introduced an extreme as that record would have become the youngest resort in the dataset.

State-wide summary statistics were derived for the market segment to investigate if supply and demand played a justifiable role to be factored into the pricing strategy. We chose to summarize the following features for state wide analysis:
- `TerrainParks`: count of the number of terrain parks at the resort
- `SkiableTerian_ac`: Total skiable area in square acres
- `daysOpenLastYear`: Total number of days open last year
- `NightSkiing_ac`: Total skiable area covered in lights for night skiing

A separate sub dataframe was created for this, counting and summing each feature grouped by state.

Finally, we dropped the records (16%) with null values for our target feature.  

### Exploratory Data Analysis (EDA)

We investigated the state-wide summary statistics combined with some resort features using principal component analysis (`PCA`). This reduced the dimension of the features of this separate sub dataframe into the top 2 `components_`, which would be able to be represented on a 2D plot. 

For PCA analysis, the data was first scaled so that each feature is standardized to have a mean of 0 and unit standard deviation. The cumulative `explained_variance_ratio_` of the `PCA` object, showed that the these top 2 `components_` accounted for more than 75% of the variance in this sub dataframe.

![Plot showing the cummulative explained variance ratio explained by the PCA components for the state/resort summary statistics with the first 2 components summing to more than 75% cumulatively](../images/cumulative_variance_ratio_states.svg)

An xy plot of the first PCA component against the 2nd PCA component annotated by state, and further dimensionalize to include the mean `AdultWeekdend` ticket price by state (binned into `Quartile`s and represented using `hue` (color) on the plot) did not demonstrate any obvious distinction for categorizing `AdultWeekend` ticket price based on the `state`.

![x-y plot of the first component analysis against the second component analysis annotates to show the state abbreviation and point sized according to the mean adult weekend ticket price](../images/pca_components_state_AdultWeekend.svg)

We learned from these which resort/state-wide features we had tended to vary the most among states but these variations were not parallel with the price patterns and showed no trend. From these initial explorations, we considered it justifiable to not consider statelines as a boundary, thereby to not consider the `state` feature in the model, and to treat all resorts the same no matter its `state` value.

We also further explored a few new features (feature engineering) by combining the 'resort-level data' with some of the state-wide features in ratios we thought may follow the `AdultWeekend` ticket prices. No new feature which included the state info had a high correlation coefficient with the target feature which gave further validity to not consider the `state` feature. The numerical features which had the highest Pearson correlation with `AdultWeekend` were 
- `Runs`,
- `fastQuads`,
- `vertical_drop`,
- `Snow Making_ac`, and
- `total_chairs`,

each with a correlation coefficient of 0.65 or higher.

![Colormap showing the correlation coefficient of the numerical features with the Adult weekend ticket price.](../images/corr_coef_AdultWeekend.png)





### Model Preprocessing and Feature engineering
Having some insight for which features we wanted to include in our model, we took steps towards building a model. Our initial model sometimes referred to as a dummy model would use only the mean for predictions.

To measure the performance of the model we use the **coefficient of determination** also referred to as the **r-squared ($R^2$)** value. Simply put,  this is a measure of the goodness of fit. It usually ranges between 0 and 1, with our dummy model which only predicts the mean giving a $R^2$ value 0. By this way, we know that if we get a $R^2$ value less than 0, we are preforming worse than our dummy model. We also use the more intuitive  **mean absolute error ($MAE$)** and the **root mean square error ($RSME$)** as metrics. 

Before any preprocessing is done we removed `Big Mountain Resort` (BMR) record from the dataset as we want to predict a price based on BMR's feature values. We had split the data into a train dataset and a test dataset before any preprocessing to prevent leakage, and only used the train dataset to fit the model. The test dataset was used to evaluate model performance and check for overfitting. Given that the dataset was small, we used cross-validation to improve our confidence as we assessed the model's performance.

We used a number of modules from the `scikit-learn`'s library to streamline the model testing process. For missing values we used `scikit-learn`'s `SimpleImputer`. `SimpleImputer` was used to fill in the mean or the median feature value for the missing/null values. This preprocessing step is necessary as our model would not accept null values.

To organize the model preprocessing steps we used `scikit-learn`'s `Pipeline` module to build a `pipeline` object. A `pipeline` prevented us from 'leaking data' from the test set into the model during training and also allowed us to insert other transforms/ preprocessing steps and a final estimator/model object into the `pipeline` itself and wrap the entire process in one object. This greatly streamline the model building/determination process and significantly reduces errors while testing models with various combinations of hyperperameters, as a `pipeline` also accepts the same characteristic `.fit()` and `.predict()` method as other `scikit-learn` estimators.

### Algorithms

We created a pipeline which 
- imputed  missing values using `SimpleImputer` with the median value. Initial trial had no difference between imputation with mean vs median.
- scales the data using `StandardScaler`
- selects the best k features using `SelectKBest`. 
- train and fit data to a `LinearRegression` model.
- assessed the pipeline performance using 'cross-validation'. 
  
Cross validation partitions the training set into k-folds, and trains on all except 1 of these folds, and uses that 1 fold to test performance on the training data. As it cycles through each of the folds  in the train dataset to estimate model performance, it is able to capture any variability  which may be latent be present from a single split.

We soon notice that our model's performance varied according to what value we choose for `k` parameter in the `SelectKBest` object. This was in addition to the performance variance that was also present from the `cross_validate` folds. In order to select the best `k` for our `LinearRegression()` model we used `scikit-learn`'s `GridSearchCV`. This further accelerated the model selection process by cycling through an array of different values for `k`, allowing us to quickly see how the model performance varied with `k`, and the option to automatically select the `best_params_` . Our metric here was $R^2$ score. The plot below shows that this occured when `k = 8`.

![Plot showing the model's performance at different k values from GridSearchCV](../images/cv_scores_selectkbest.svg)


We can see here in the plot ab0ove that our $R^2$ score is `0.66` which is significantly higher than our baseline score of `0.0`. 

### Winning Model
An algorithm known to work quite well in many situations is Random Forest. With our regression problem we are able to utilize `scikit-learn`'s `RandonForestRegressor` class. Similar to prior, we create a pipeline which uses
- `SimpleImputer` with median values
- `StandardScaler`
- `RandomForestRegressor` with default parameters. 

We get a mean cross_val $R^2$ score of `0.70`. which is a slight improvement over the previous `LinearRegression` model.  As before, we perform a `GridSearchCV` varying the `n_estimators`, together with the all the other hyper-perameters available in the pipeline, but scoring using $MAE$ instead of $R^2$ for a direct price comparison. 

Our `best_params_` for our `best_estimator_` were:
- `n_estimators`: `46` 
- `simpleimputer__strategy`: `median`
- `standardscaler`: `None`

This is not surprising as these are similar properties to the `LinearRegression` model with the exception of not scaling but that is expected as the Random Forest algorithm is based on partitioning the data to make predictions therefore it does not require normalization. Like other tree based models, we are able to extract the `features_importances_` or 'Gini importance'.

!['Plot showing Feature importances form the best Random Forest Regressor model](../images/rf_feature_importances_hbar.svg)

Encouragingly, the top 5 Feature importance were the same numerical features which had the highest Pearson correlation coefficient with `AdultWeekend` highlighted above. 

Using our best  `RandomForestRegressor` estimator we fit the model to the training data using cross validation and got a 
mean absolute error of `9.71` on the train dataset and
`9.64` on the test dataset.

We also tested our `LinearRegression` model for comparison and it scored
a mean absolute error of `10.50` on the train dataset and
`11.79` on the test dataset. 




### Price Recommendation

Using our best estimator, we fit our data one more time on all the resort data we have, excluding BMR, to then estimates an `AdultWeekend` ticket price for BMR based on BMR present features. Our model predicted BMR to have a ticket price of `$98.40`. That is `$17.40` higher than BMR present price, and even in excess of the `$10.20` mean absolute error score of our model. Since this model predicts BMR to have a higher ticket price, we recommend increasing prices. A ticket price of `$88.20` would be within our margin of error.

### Conclusion

This project set out to build a data driven model to predict the ticket price for BMR based on the features and facilities at this resort. The model improved upon the former strategy of basing prices on just the market average, and is able to predict prices even beyond present features, based on hypothetical scenarios. The `RandomForestRegressor` model performed the best out of the models we trained and tested and so is our winning model. It predicted a higher price than what is presently charged for the target `AdultWeekend` ticket and so we recommend the BMR raises ticket prices to capitalize more at its facilities.



### Future Work

There are a few improvements possible to increase the accuracy of the present `RandomForestRegressor` best estimator. Some Recommendations are:
- More fine tuning of the hyper-parameters
- modelling with less features and removing the redundant features
- include other revenue other than the ticket price.
- We only looked at a snapshot. Prior grow and price changes can help.
- demographics of who the BMR's patrons are can give an pointed view of the present market that they capture. These include age groups, skill levels, business vs personal.
- dividing into geographical regions. State lines did not seem to be a crucial feature, but there may be other boundaries to be considered.
- Nontangible feature such as influencers were not considered.